# Notebook 5: Making Decisions And Adaptive Acquisitions

Decisions are a type of object that takes in Processed Data and returns a suggested Acquisition which will be acquired.



In [1]:
import sys,os
sys.path.append('source')

  ## Example 5.1: Making A decision that repeats the previous acquisition.

  Decisions follow a similar structure to Image Processing.

In [4]:
from source.decision import *

data=[]
acq=AcquisitionPlugin()

decision=DecisionNull()
new_acq=decision.propose(data,acq)

decision=DecisionRepeatAcquisition()
new_acq=decision.propose(data,acq)
DecisionThreshHoldGreaterThan(key,value)
DecisionThreshHoldLessThan(key,value)

In [ ]:
decision=DecisionRepeatAcquisition()
new_acq=decision.propose(data,acq)